RRN WITH ATTENTION

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np 
import torch.optim as optim
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
import warnings 
warnings.filterwarnings("ignore")

In [2]:
def get_fib_seq(n,scaled_data=True):
    seq = np.zeros(n)

    fib_n1 = 0.0
    fib_n = 1.0

    for i in  range(n):
        seq[i] = fib_n1 + fib_n
        fib_n1 = fib_n
        fib_n = seq[i]
    
    scaler = MinMaxScaler(feature_range=(0,1)) if scaled_data else None
    if scaled_data:
        seq = scaler.fit_transform(seq.reshape(-1,1)).flatten()
    
    return seq ,scaler

In [3]:
seq , scaler = get_fib_seq(10)
print(seq)
# print(scaler)

[0.         0.01136364 0.02272727 0.04545455 0.07954545 0.13636364
 0.22727273 0.375      0.61363636 1.        ]


In [4]:

def get_fib_XY(total_fib_numbers, time_step, train_percent, scaled_data=True):
    dat, scaler = get_fib_seq(total_fib_numbers, scaled_data)
    Y_ind = np.arange(time_step, len(dat), 1)
    

    Y = dat[Y_ind]
    rows_x = len(Y_ind)
    print("ROWS_X \n", rows_x, "\n")

    X = np.array([dat[i:rows_x + i] for i in range(time_step)]).T

    rand = np.random.RandomState(seed=13)
    idx = rand.permutation(rows_x)
    
    split = int(train_percent * rows_x)

    train_ind, test_ind = idx[:split], idx[split:]
    trainX = X[train_ind]
    trainY = Y[train_ind]
    testX = X[test_ind]
    testY = Y[test_ind]
    
  
    trainX = trainX.reshape(-1, time_step, 1)
    testX = testX.reshape(-1, time_step, 1)
    

    trainX = torch.tensor(trainX, dtype=torch.float32)
    trainY = torch.tensor(trainY, dtype=torch.float32)
    testX = torch.tensor(testX, dtype=torch.float32)
    testY = torch.tensor(testY, dtype=torch.float32)

    print(f"TrainX{trainX.shape} \n TrainY{trainY.shape}")
    
    return trainX, trainY, testX, testY


###### Before creating RNN things to remember
###### The RNN model expects the input to have a 3d shape
###### i.e (batchsize,sequence_lenght,input_size) we have the sequence lenght and input size 
###### we need a  batch  dimension so  we use unsqueeze function  from  pytorch  to  add  it 
###### and  same we  need  a batch dimesnion  for y  while  we  pass it in  criterion function 
###### inshort we do it for the rnn model compatibility 

In [5]:
rd_tensor = [0,0.01136364,0.02272727 ,0.04545455 ,0.07954545 ,0.13636364,0.22727273, 0.375 ,0.61363636,1]
rd_tensor = torch.tensor(rd_tensor)
print(f"TENSOR SHAPE {rd_tensor.shape}")
print(f"adding batch  dimension {rd_tensor.unsqueeze(0).shape}")

TENSOR SHAPE torch.Size([10])
adding batch  dimension torch.Size([1, 10])


Creating simple RNN

In [6]:
class simpleRNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(simpleRNN,self).__init__()
        self.rnn = nn.RNN(input_size,hidden_size,batch_first=True)
        self.fc = nn.Linear(hidden_size,output_size)

    def forward(self,x):
        out,_ = self.rnn(x)
        out = self.fc(out[:,-1,:])
        return out
    
print("SIMPLE RNN MODEL")

SIMPLE RNN MODEL


In [7]:
time_steps = 20
hidden_size = 2
epochs = 30
train_percent = 0.8
total_fib_numbers = 1200

trainX,trainY,testX,testY = get_fib_XY(total_fib_numbers,time_steps,train_percent)
model = simpleRNN(input_size=1,hidden_size=hidden_size,output_size=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)


for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for i  in tqdm(range(len(trainX)),desc=f"EPOCH {epoch + 1}/{epochs}"):
        optimizer.zero_grad()
        outputs = model(trainX[i].unsqueeze(0)) # adding batch dimension
        loss = criterion(outputs,trainY[i].unsqueeze(0))

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(trainX):.4f}")

# Evaluation
model.eval()
train_mse = criterion(model(trainX), trainY).item()
test_mse = criterion(model(testX), testY).item()

print("Test set MSE =", test_mse)
print("Train set MSE =", train_mse)

ROWS_X 
 1180 

TrainXtorch.Size([944, 20, 1]) 
 TrainYtorch.Size([944])


EPOCH 1/30: 100%|██████████| 944/944 [00:00<00:00, 1153.03it/s]


Epoch [1/30], Loss: 0.0017


EPOCH 2/30: 100%|██████████| 944/944 [00:00<00:00, 1163.85it/s]


Epoch [2/30], Loss: 0.0016


...

Epoch [28/30], Loss: 0.0007


EPOCH 29/30: 100%|██████████| 944/944 [00:00<00:00, 1173.07it/s]


Epoch [29/30], Loss: 0.0006


EPOCH 30/30: 100%|██████████| 944/944 [00:00<00:00, 1150.84it/s]


Epoch [30/30], Loss: 0.0005
Test set MSE = 0.001966223120689392
Train set MSE = 0.001585272024385631
